In [24]:
import os
import cv2

from sklearn.model_selection import train_test_split

import numpy as np

from PIL import Image

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [25]:
# Cargar el conjunto de datos de manzanas, peras, bananos y fresas

# Directorio donde se encuentran las imágenes de bananos
banano_dir = 'frutas_dataset/bananos/subcarpeta1'

# Lista para almacenar las imágenes de bananos
banano_images = []

# Recorrer los archivos en el directorio
for filename in os.listdir(banano_dir):
    # Comprobar si el archivo es una imagen
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Ruta completa del archivo de imagen
        img_path = os.path.join(banano_dir, filename)
        # Leer la imagen utilizando OpenCV
        img = cv2.imread(img_path)
        # Agregar la imagen a la lista
        banano_images.append(img)

# Imprimir el número de imágenes de bananos cargadas
print("Número de imágenes de bananos cargadas:", len(banano_images))


Número de imágenes de bananos cargadas: 490


In [26]:
# Dividir los datos en conjuntos de entrenamiento y prueba
train_images, test_images = train_test_split(banano_images, test_size=0.2, random_state=42)

# Imprimir el número de imágenes en cada conjunto
print("Número de imágenes de bananos en el conjunto de entrenamiento:", len(train_images))
print("Número de imágenes de bananos en el conjunto de prueba:", len(test_images))

Número de imágenes de bananos en el conjunto de entrenamiento: 392
Número de imágenes de bananos en el conjunto de prueba: 98


In [27]:
# Función para preprocesar una imagen de banano
def preprocess_image(image):
    # Redimensionar la imagen a un tamaño específico
    resized_image = image.resize((224, 224))
    
    # Normalizar los valores de píxel entre 0 y 1
    normalized_image = np.array(resized_image) / 255.0
    
    # Cambiar el orden de los canales de BGR a RGB (si es necesario)
    rgb_image = normalized_image[:, :, ::-1]
    
    return rgb_image

# Preprocesar las imágenes de bananos en el conjunto de entrenamiento
preprocessed_train_images = []
for image in train_images:
    pil_image = Image.fromarray(image)
    preprocessed_image = preprocess_image(pil_image)
    preprocessed_train_images.append(preprocessed_image)

# Preprocesar las imágenes de bananos en el conjunto de prueba
preprocessed_test_images = []
for image in test_images:
    pil_image = Image.fromarray(image)
    preprocessed_image = preprocess_image(pil_image)
    preprocessed_test_images.append(preprocessed_image)
    
print("Conjunto de entrenamiento:", len(preprocessed_train_images))
print("Conjunto de prueba:", len(preprocessed_test_images))

Conjunto de entrenamiento: 392
Conjunto de prueba: 98


In [28]:
# Construcción del modelo de la CNN

# Crear el modelo
model = Sequential()

# Agregar capas de convolución y max pooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Aplanar la salida de la última capa convolucional
model.add(Flatten())

# Agregar capas densas (fully connected)
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))  # 4 clases: manzanas, peras, bananos, fresas

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Imprimir un resumen del modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [ ]:
# Paso 3: Entrenamiento del modelo

model.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_images, test_labels))

# Paso 4: Evaluación del modelo

test_loss, test_accuracy = model.evaluate(test_images, test_labels)

# Paso 5: Clasificación de nuevas imágenes

# Descargar imágenes de frutas de Internet
# Preprocesar las imágenes
# Utilizar el modelo entrenado para predecir las clases de las nuevas imágenes